In [ ]:
# Verification de l'etat des authorisation Inbound et Outbound d'un Workspace
from notebookutils import mssparkutils
import requests


WID = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"

# Recuperation du token
token = mssparkutils.credentials.getToken("https://api.fabric.microsoft.com")

# Contruction du header
h = {"Authorization": f"Bearer {token}", "Content-Type":"application/json"}

# Test GET (reponse 200 attendue)
get_request = requests.get(f"https://api.fabric.microsoft.com/v1/workspaces/{WID}/networking/communicationPolicy", headers=h)
print(get_request.status_code, get_request.text)




In [ ]:
# Modification des authorisation Inbound et Outbound d'un Workspace
from notebookutils import mssparkutils
import requests, json, time

WID = "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"

# Valeur cible Inbound defaultvalue
target_inbound_value = "Deny"

# Recuperation du token Fabric API
token = mssparkutils.credentials.getToken("https://api.fabric.microsoft.com")

# Contruction du header
h = {"Authorization": f"Bearer {token}", "Content-Type":"application/json"}

# Recupèration de la policy actuelle
current_policy = requests.get(
    f"https://api.fabric.microsoft.com/v1/workspaces/{WID}/networking/communicationPolicy", 
    headers=h
    ).json()

print("Policy actuelle: ", current_policy)

# Contruction de la policy avec Inbound = Deny
new_policy = current_policy
new_policy.setdefault("inbound", {}).setdefault("publicAccessRules", {})["defaultAction"] = target_inbound_value

# Application de la policy 
put_request = requests.put(
    f"https://api.fabric.microsoft.com/v1/workspaces/{WID}/networking/communicationPolicy", 
    headers=h,
    data=json.dumps(new_policy)
)
print("PUT Status", put_request.status_code, put_request.text)

# Boucle pour attendre la maj
for _ in range(4):
    time.sleep(1)
    test_result = requests.get(
        f"https://api.fabric.microsoft.com/v1/workspaces/{WID}/networking/communicationPolicy",
         headers=h
    ).json()
    print("defaultAction:", test_result)
    if test_result.get("inbound", {}).get("publicAccessRules", {}).get("defaultAction") == target_inbound_value:
        break

